In [1]:
from local_llm_judge.compare import stag_vs_prod

queries = ["adidas sambas", "red adidas sambas", "vegan leather jacket",
           "clutch purse", "blue clutch purse"]


results = stag_vs_prod(queries)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loaded 41 results
Skipping 4 queries - ['adidas sambas' 'red adidas sambas' 'clutch purse' 'blue clutch purse']
Comparing 1 queries


In [4]:
import pandas as pd

def _url_to_img_tag(path):
    return f"""<img
    style="width: 100px; height: 100px; object-fit: contain;"
            src="{path}"/>"""


def styled(df: pd.DataFrame) -> pd.DataFrame:
    return df.style.format({'image_url_lhs': _url_to_img_tag,
                            'image_url_rhs': _url_to_img_tag})


styled(results)

,query,name_lhs,option_id_lhs,image_url_lhs,prob_lhs,name_rhs,option_id_rhs,image_url_rhs,prob_rhs
0,adidas sambas,Samba 'Valentine's Day' sneakers,e91f6777-b005-443f-a8c5-f1d6895866e6,,0.401786,SAMBAE SNEAKER,8a69cc90-83ae-4cba-9fa7-f155356da7da,,0.598214
1,adidas sambas,Sambae Sneaker at Nordstrom,4eed2b40-848a-459c-aceb-a17808695d06,,0.494145,SAMBAE SNEAKER,0b3f3351-d993-4778-b7d3-60e7f7f75ce8,,0.505855
2,adidas sambas,Sambae Sneaker at Nordstrom,3ec6ac9e-2490-4068-9b08-227dcdd6967a,,0.494145,"SAMBA OG SNEAKER ,.5",5eb39e1e-edc1-49e7-b4a8-03d3c4218e80,,0.505855
3,adidas sambas,"Samba Sneaker at Nordstrom,.5",0cd130f8-dd23-4afc-8c25-7f28bee43ced,,0.641509,SAMBAE SNEAKER,95860adb-b34c-46e9-bf5d-8210f7c303f4,,0.358491
4,adidas sambas,"Samba Sneaker at Nordstrom,.5",a1b50f9c-4572-4e4c-92f3-17bc93565821,,0.641509,Samba 'Valentine's Day' sneakers,50121353-94b2-4e3b-babb-4cdf91fbba1a,,0.358491
5,adidas sambas,Samba LT Sneaker at Nordstrom,a6ac07da-0547-429b-b70e-b45594c53091,,0.494145,SAMBA SNEAKER,1e4c3e90-633f-4ce0-a455-6b30ca20b527,,0.505855
6,adidas sambas,Sambae Sneaker at Nordstrom,b69be293-6038-4bfb-a671-38113eb878e8,,0.494145,SAMBAE SNEAKER,82a762e5-e105-4916-9c5a-220e625f1068,,0.505855
7,adidas sambas,Sambae Sneaker at Nordstrom,fbded8da-a69a-472f-8546-8a34a7a457a3,,0.494145,SAMBAE SNEAKER,35cea507-e413-4f20-88f7-3a2064d10c87,,0.505855
8,adidas sambas,Sambae Sneaker at Nordstrom,7cc4264c-556e-463c-a1e8-0cc19a87af3d,,0.494145,SAMBAE SNEAKER,8c6b413f-79ee-4469-af08-be311a3f3f83,,0.505855
9,adidas sambas,Samba Sneaker at Nordstrom,e3e52a6a-0b55-4880-83ed-f39bf7177859,,0.494145,SAMBA LT SNEAKER,691c8ebf-de21-40cc-ae0f-1d455666d078,,0.505855
